In [ ]:
import pandas as pd
import xarray as xr
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit

from seapopym_optimization.constraint.energy_transfert_constraint import EnergyCoefficientConstraint
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.functional_group.base_functional_group import FunctionalGroupSet
from seapopym_optimization.model_generator import NoTransportModelGenerator


In [ ]:
fg1 = NoTransportFunctionalGroup(
    name="FG1",
    day_layer=0,
    night_layer=0,
    energy_transfert=Parameter(name="FG1_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

fg2 = NoTransportFunctionalGroup(
    name="FG2",
    day_layer=0,
    night_layer=1,
    energy_transfert=Parameter(name="FG2_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

In [ ]:
constraint = EnergyCoefficientConstraint(
    parameters_name=["FG1_energy_coefficient", "FG2_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

In [ ]:
fg_set = FunctionalGroupSet(
    functional_groups=[fg1, fg2],
)
functional_group_parameters = fg_set.generate([1, 1, 1, 1, -0.5, 1])

In [ ]:
temperature = xr.DataArray(
    data=[[[1]], [[2]], [[3]], [[4]], [[5]]],
    dims=["time", "latitude", "longitude"],
    coords={
        "time": pd.date_range("2023-01-01", periods=5, freq="D"),
        "latitude": [0],
        "longitude": [0],
    },
    name="temperature",
    attrs={
        "units": "Celsius",
        "long_name": "Sea surface temperature",
        "standard_name": "sea_surface_temperature",
    },
)
primary_production = xr.DataArray(
    data=[[[1]], [[2]], [[3]], [[4]], [[5]]],
    dims=["time", "latitude", "longitude"],
    coords={
        "time": pd.date_range("2023-01-01", periods=5, freq="D"),
        "latitude": [0],
        "longitude": [0],
    },
    name="primary_production",
    attrs={
        "units": "g/m^2/day",
        "long_name": "Primary production",
        "standard_name": "primary_production",
    },
)
temperature.time.attrs = {"axis": "T"}
primary_production.time.attrs = {"axis": "T"}
temperature.latitude.attrs = {"axis": "Y"}
primary_production.latitude.attrs = {"axis": "Y"}
temperature.longitude.attrs = {"axis": "X"}
primary_production.longitude.attrs = {"axis": "X"}

forcing_parameter = ForcingParameter(
    temperature=ForcingUnit(forcing=temperature),
    primary_production=ForcingUnit(forcing=primary_production),
)

In [ ]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameter)

In [ ]:
model = model_generator.generate(functional_group_parameters=functional_group_parameters)